<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/UnifiedLLM_Embeded_EFT%2Bunsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install torchviz torchsummary hiddenlayer

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4132 sha256=a580784d37ff9182e09072514a999b7a7dfd709c6b5e5a2c620f6b31ce359446
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [9]:
!pip install torch transformers tqdm datasets

In [10]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Data처리

In [11]:
from datasets import load_dataset, ClassLabel, DatasetDict
from sklearn.model_selection import train_test_split

agnews_dataset = load_dataset("fancyzhx/ag_news")
existing_labels = agnews_dataset["train"].features["label"].names
print(f"AG News labels: {existing_labels}")

# 새로운 레이블 생성 - 'StackOverflow' 추가
new_labels = existing_labels + ["StackOverflow"]
new_class_label = ClassLabel(names=new_labels)

# AG News 데이터셋의 'label' 칼럼을 새로운 ClassLabel로 캐스팅
agnews_dataset = agnews_dataset.cast_column("label", new_class_label)

# AG News 'train' 데이터를 'train'과 'validation'으로 분할
# 'train' 데이터를 90%는 'train', 10%는 'validation'으로 분할
agnews_train_valid = agnews_dataset['train'].train_test_split(test_size=0.1, seed=42)
agnews_train_dataset = agnews_train_valid['train']
agnews_validation_dataset = agnews_train_valid['test']
# 분할된 데이터를 포함하도록 agnews_dataset 업데이트
agnews_dataset = DatasetDict({
    'train': agnews_train_dataset,
    'validation': agnews_validation_dataset,
    'test': agnews_dataset['test']
})

# 결과 출력
print(agnews_dataset)

# 스플릿별 레이블 데이터 건수 카운트
label_counters = {}

for split in agnews_dataset:
    label_counts = agnews_dataset[split].features['label'].num_classes
    label_counter = {label: 0 for label in range(label_counts)}

    for example in agnews_dataset[split]:
        label_counter[example['label']] += 1

    label_counters[split] = label_counter

# 스플릿별 레이블 카운트 출력
print(label_counters)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

AG News labels: ['World', 'Sports', 'Business', 'Sci/Tech']


Casting the dataset:   0%|          | 0/120000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})
{'train': {0: 26991, 1: 26966, 2: 27100, 3: 26943, 4: 0}, 'validation': {0: 3009, 1: 3034, 2: 2900, 3: 3057, 4: 0}, 'test': {0: 1900, 1: 1900, 2: 1900, 3: 1900, 4: 0}}


In [12]:
from datasets import load_dataset, ClassLabel, concatenate_datasets, DatasetDict, Features, Value, Sequence

stackoverflow_dataset = load_dataset(
    'csv',
    data_files={
        'train': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_training_samples.csv',
        'validation': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_validation_samples.csv',
        'test': '/content/drive/MyDrive/ML/AI-algorithm/project/rev_tag_test_samples.csv'
    }
)

# StackOverflow 데이터셋 처리 함수 정의
def process_stackoverflow(example):
    title = example.get('Title', '')
    body = example.get('Body', '')
    text = f"{title} {body}".strip()
    label = new_class_label.str2int('StackOverflow')
    sub_label = example.get('Tags_new', '')
    return {'text': text, 'label': label, 'sub_label': sub_label}

# 불필요한 컬럼 제거 및 'text', 'label', 'sub_label' 적용
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].map(
        process_stackoverflow,
        remove_columns=['Title', 'Body', 'Tags_filtered', 'Tags_list', 'Algorithms', 'Backend', 'Data Science', 'Databases', 'Dev Tools', 'Frontend', 'Mobile', 'Systems', 'iOS/macOS', 'Tags_new']
    )

# 고유 태그 목록 생성 및 인덱스 부여
unique_tags = set()
for split in stackoverflow_dataset.keys():
    for example in stackoverflow_dataset[split]:
        for tag in eval(example['sub_label']):  # 문자열을 리스트로 변환
            unique_tags.add(tag)

unique_tags = sorted(list(unique_tags))  # 태그를 고유 인덱스로 정렬
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}
class_labels = ClassLabel(num_classes=len(unique_tags), names=unique_tags)

# sub_label을 멀티라벨 형식의 인덱스 리스트로 변환
def encode_sub_label(example):
    sub_labels = eval(example['sub_label'])  # 문자열을 리스트로 변환
    stack_labels = [tag2id[tag] for tag in sub_labels]  # 인덱스로 변환
    example['stack_labels'] = stack_labels  # 멀티라벨 컬럼 추가
    return example

# stack_labels 컬럼 추가
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].map(encode_sub_label)

# 'sub_label' 컬럼 제거
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].remove_columns(['sub_label'])

# StackOverflow 데이터셋의 피처를 AG News와 동일하게 설정하되, sub_label을 멀티라벨로 설정
for split in stackoverflow_dataset.keys():
    stackoverflow_dataset[split] = stackoverflow_dataset[split].cast(
        Features({
            'text': Value('string'),
            'label': new_class_label,
            'stack_labels': Sequence(class_labels)  # 멀티라벨을 위한 Sequence(ClassLabel)
        })
    )

print(stackoverflow_dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6500 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/20000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6500 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 6500
    })
    test: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 3000
    })
})


In [13]:
from datasets import Dataset, DatasetDict
from datasets import load_dataset, concatenate_datasets
from datasets import ClassLabel, Value

# agnews 데이터셋의 'stack_labels' 컬럼을 Sequence(ClassLabel)로 추가
class_labels = ClassLabel(num_classes=len(unique_tags), names=unique_tags)

def add_empty_stack_labels(example):
    example['stack_labels'] = []  # 빈 리스트로 초기화
    return example

for split in agnews_dataset.keys():
    agnews_dataset[split] = agnews_dataset[split].map(add_empty_stack_labels)

# AG News 데이터셋의 피처를 StackOverflow 데이터셋과 동일하게 설정
for split in agnews_dataset.keys():
    agnews_dataset[split] = agnews_dataset[split].cast(
        Features({
            'text': Value('string'),
            'label': new_class_label,
            'stack_labels': Sequence(class_labels)  # 빈 Sequence(ClassLabel)로 설정
        })
    )


# 4. 각 스플릿별로 데이터셋 결합
combined_splits = {}
for split in agnews_dataset.keys():
    if split in stackoverflow_dataset:
        combined_splits[split] = concatenate_datasets(
            [agnews_dataset[split], stackoverflow_dataset[split]]
        )
    else:
        combined_splits[split] = agnews_dataset[split]

# 결합된 데이터셋 생성
combined_dataset = DatasetDict(combined_splits)

# 결과 확인
print(combined_dataset)

# 스플릿별 레이블 데이터 건수 카운트
label_counters = {}

for split in combined_dataset:
    label_counts = combined_dataset[split].features['label'].num_classes
    label_counter = {label: 0 for label in range(label_counts)}

    for example in combined_dataset[split]:
        label_counter[example['label']] += 1

    label_counters[split] = label_counter

print(label_counters)


Map:   0%|          | 0/108000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/108000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 128000
    })
    validation: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 10600
    })
})
{'train': {0: 26991, 1: 26966, 2: 27100, 3: 26943, 4: 20000}, 'validation': {0: 3009, 1: 3034, 2: 2900, 3: 3057, 4: 6500}, 'test': {0: 1900, 1: 1900, 2: 1900, 3: 1900, 4: 3000}}


In [6]:
# 5. 레이블별 데이터 건수 출력
from collections import Counter

for split in combined_dataset.keys():
    labels = combined_dataset[split]['label']
    label_counts = Counter(labels)
    print(f"\n{split} 스플릿의 레이블 분포:")
    for label_int, count in label_counts.items():
        label_name = new_class_label.int2str(label_int)
        print(f"{label_name}: {count}")


train 스플릿의 레이블 분포:
World: 26991
Sci/Tech: 26943
Sports: 26966
Business: 27100
StackOverflow: 20000

validation 스플릿의 레이블 분포:
World: 3009
Sports: 3034
Sci/Tech: 3057
Business: 2900
StackOverflow: 6500

test 스플릿의 레이블 분포:
Business: 1900
Sci/Tech: 1900
Sports: 1900
World: 1900
StackOverflow: 3000


In [6]:
#### Dataset 확인
# 'Sci/Tech' 레이블의 하나의 데이터 출력
sci_tech_label_id = new_class_label.str2int('Sci/Tech')

# 각 스플릿에서 'Sci/Tech' 레이블을 가진 예제 찾기
for split in combined_dataset.keys():
    sci_tech_examples = combined_dataset[split].filter(lambda example: example['label'] == sci_tech_label_id)
    if len(sci_tech_examples) > 0:
        print(f"'Sci/Tech' 레이블의 예제 (스플릿: {split}):")
        print(sci_tech_examples[0])
        break

# 'StackOverflow' 레이블의 하나의 데이터 출력
stackoverflow_label_id = new_class_label.str2int('StackOverflow')

# 각 스플릿에서 'StackOverflow' 레이블을 가진 예제 찾기
for split in combined_dataset.keys():
    stackoverflow_examples = combined_dataset[split].filter(lambda example: example['label'] == stackoverflow_label_id)
    if len(stackoverflow_examples) > 0:
        print(f"\n'StackOverflow' 레이블의 예제 (스플릿: {split}):")
        print(stackoverflow_examples[1])
        break


'Sci/Tech' 레이블의 예제 (스플릿: train):
{'text': 'Bob Evans, mainframe pioneer, dies at 77 Evans led a team that developed a new class of mainframe computers that allowed different applications to be run simultaneously.', 'label': 3, 'stack_labels': []}

'StackOverflow' 레이블의 예제 (스플릿: train):
{'text': 'pointer from one class to anotehr Hello I want to point "seller" to "name"\r\nbut it gives me an error. please help. \r\n\r\n`\r\nclass Person {\r\n    public:\r\n        friend class Apartment;\r\n    \tPerson() {\r\n    \t  cout << "~ ";\r\n    \t}//constructor\r\n        string name = "Mr.Smith\\n";\r\n        string TelNumber = "01";\r\n        int age = 0;\r\n};\r\n\r\n\r\nclass Apartment {\r\n\tpublic:\r\n\r\n\r\n\t\tstring Address = "jyu";\r\n\t\tdouble Area = 74.0;\r\n\t\tdouble Price = 320.0;\r\n\t\tint numberOfRooms = 3;\r\n\t\t//string name = "d";\r\n\t\tstring *Seller = &name;\r\n\t\t//Apartment(string *Seller = null) {\r\n\r\n\t\t//}\r\n\t\t// constructor for seller\r\n\t\t Apartmen

In [14]:
# combined_dataset의 각 스플릿을 변수에 저장
combined_train_dataset = combined_dataset['train']
combined_validation_dataset = combined_dataset['validation']
combined_test_dataset = combined_dataset['test']

# 새로운 DatasetDict 생성
new_agnews_dataset = DatasetDict({
    "train": combined_train_dataset,
    "validation": combined_validation_dataset,
    "test": combined_test_dataset
})

# 결과 확인
print(new_agnews_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 128000
    })
    validation: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['text', 'label', 'stack_labels'],
        num_rows: 10600
    })
})


In [ ]:
# from transformers import AutoTokenizer

# new_agnews_dataset= DatasetDict({
#     "train": combined_train_dataset,
#     "test": combined_test_dataset
# })

# # DatasetDict 출력
# print(new_agnews_dataset)

# tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)

# tokenized_datasets = new_agnews_dataset.map(tokenize_function, batched=True)



# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1500))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1500))

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9500
    })
})


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9500 [00:00<?, ? examples/s]

# **1. Llama3.2-3B Embedding**

In [15]:
import os
import torch
import json
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from transformers import LlamaForCausalLM, LlamaTokenizer
from tqdm import trange
from datasets import load_dataset
import pandas as pd

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [50]:
# # -*- coding: utf-8 -*-

# def rep_extract(task, mode, device, sents, stack_labels, labels, max_len, step):
#     model_id = "unsloth/Llama-3.2-3B-Instruct"

#     tokenizer = AutoTokenizer.from_pretrained(model_id)

#     # padding 토큰 설정 (eos_token을 pad_token으로 설정)
#     if tokenizer.pad_token is None:
#         tokenizer.pad_token = tokenizer.eos_token

#     tokenizer.padding_side = "right"

#     config_kwargs = {
#         "trust_remote_code": True,
#         "cache_dir": None,
#         "revision": 'main',
#         "use_auth_token": None,
#         "output_hidden_states": True
#     }
#     model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
#     model = LlamaForCausalLM.from_pretrained(
#         model_id,
#         trust_remote_code=True,
#         config=model_config,
#         device_map=device,
#         torch_dtype=torch.float16
#     )
#     model.eval()

#     sents_reps = []
#     for idx in trange(0, len(sents), step):
#         idx_end = idx + step
#         if idx_end > len(sents):
#             idx_end = len(sents)

#         # sents_batch와 stack_labels 결합
#         sents_batch = sents[idx: idx_end]
#         stack_labels_batch = stack_labels[idx: idx_end]
#         combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

#         sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
#         sents_batch_encoding = sents_batch_encoding.to(device)

#         with torch.no_grad():
#             batch_outputs = model(**sents_batch_encoding)

#             reps_batch_5L = []
#             for layer in range(-1, -6, -1):
#                 reps_batch_5L.append(torch.mean(batch_outputs.hidden_states[layer], axis=1))
#             reps_batch_5L = torch.stack(reps_batch_5L, axis=1)

#         sents_reps.append(reps_batch_5L.cpu())
#     sents_reps = torch.cat(sents_reps)

#     labels = torch.tensor(labels)

#     path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
#     if not os.path.exists(path):
#         os.makedirs(path)
#     torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
#     torch.save(labels, path + f'{mode}_labels.pt')

# if __name__ == '__main__':
#     cuda_no = 0  # GPU 번호
#     task = 'llama3-2'  # Task 이름
#     device = f'cuda:{cuda_no}'
#     max_len = 512
#     step = 32

#     # 'train' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['train']['text']
#     stack_labels = combined_dataset['train']['stack_labels']
#     labels = combined_dataset['train']['label']
#     rep_extract(task, 'train', device, sents, stack_labels, labels, max_len, step)

#     # 'validation' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['validation']['text']
#     stack_labels = combined_dataset['validation']['stack_labels']
#     labels = combined_dataset['validation']['label']
#     rep_extract(task, 'validation', device, sents, stack_labels, labels, max_len, step)

#     # 'test' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['test']['text']
#     stack_labels = combined_dataset['test']['stack_labels']
#     labels = combined_dataset['test']['label']
#     rep_extract(task, 'test', device, sents, stack_labels, labels, max_len, step)


tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/928 [00:00<?, ?B/s]

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

100%|██████████| 4000/4000 [45:41<00:00,  1.46it/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 579/579 [06:36<00:00,  1.46it/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
100%|██████████| 332/332 [03:47<00:00,  1.46it/s]


In [9]:
%pip install optuna

In [9]:
# -*- coding: utf-8 -*-
import os
import torch
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from tqdm import trange

def rep_extract(task, mode, device, sents, stack_labels, labels, max_len, step):
    model_id = "unsloth/Llama-3.2-3B-Instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    config_kwargs = {
        "cache_dir": None,
        "revision": 'main',
        "use_auth_token": None,
        "output_hidden_states": True
    }
    model_config = AutoConfig.from_pretrained(model_id, **config_kwargs)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config=model_config,
        torch_dtype=torch.float16
    ).to(device)

    model.eval()

    # 최적 가중치 설정 (Trial 0의 최적 결과 사용)
    layer_weights = torch.tensor([0.157, 0.456, 0.424], device=device, dtype=torch.float16)
    layer_weights = layer_weights / layer_weights.sum()  # 가중치 정규화

    sents_reps = []
    for idx in trange(0, len(sents), step):
        idx_end = idx + step
        if idx_end > len(sents):
            idx_end = len(sents)

        sents_batch = sents[idx: idx_end]
        stack_labels_batch = stack_labels[idx: idx_end]
        combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

        sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
        sents_batch_encoding = sents_batch_encoding.to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            selected_layers = [-1, -2, -3]
            layer_embeddings = torch.stack([batch_outputs.hidden_states[layer][:, 0, :] for layer in selected_layers], dim=0)
            weighted_sum = (layer_embeddings * layer_weights.view(-1, 1, 1)).sum(dim=0)

            sents_reps.append(weighted_sum.unsqueeze(1))

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    sents_reps = torch.cat(sents_reps)
    labels = torch.tensor(labels)

    path = f'/content/drive/MyDrive/ML/AI-algorithm/project2/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps, path + f'{mode}_sents.pt')
    torch.save(labels, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # CUDA 오류 디버깅 활성화
    cuda_no = 0  # GPU 번호
    task = 'llama3-2'  # Task 이름
    device = f'cuda:{cuda_no}'
    max_len = 512
    step = 32

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['train']['text']
    stack_labels = combined_dataset['train']['stack_labels']
    labels = combined_dataset['train']['label']
    rep_extract(task, 'train', device, sents, stack_labels, labels, max_len, step)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['validation']['text']
    stack_labels = combined_dataset['validation']['stack_labels']
    labels = combined_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, stack_labels, labels, max_len, step)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['test']['text']
    stack_labels = combined_dataset['test']['stack_labels']
    labels = combined_dataset['test']['label']
    rep_extract(task, 'test', device, sents, stack_labels, labels, max_len, step)


100%|██████████| 332/332 [04:25<00:00,  1.25it/s]


In [10]:
import torch

# 파일 경로 설정
l_path = '/content/drive/MyDrive/ML/AI-algorithm/project2/llama3-2/dataset_tensor/'
mode = 'train'  # train, test, validation 등의 모드에 따라 변경 가능

# 임베딩 로드
l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
print(f"Llama Embedding Shape: {l_sents_reps.shape}")  # [128000, 5, 3072] 형태 확인

# 레이어별 통계 출력 함수 정의
def analyze_embeddings(embeddings):
    num_layers = embeddings.shape[1]  # 레이어 수 (5)
    for layer in range(num_layers):
        layer_embedding = embeddings[:, layer, :]
        layer_mean = layer_embedding.mean().item()
        layer_std = layer_embedding.std().item()
        layer_min = layer_embedding.min().item()
        layer_max = layer_embedding.max().item()

        print(f"Layer {layer + 1} Statistics:")
        print(f"  Mean: {layer_mean:.4f}")
        print(f"  Std Dev: {layer_std:.4f}")
        print(f"  Min: {layer_min:.4f}")
        print(f"  Max: {layer_max:.4f}")
        print(f"  Shape: {layer_embedding.shape}\n")

# 임베딩 분석 실행
analyze_embeddings(l_sents_reps)


<ipython-input-10-222b3654ee3c>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')


Llama Embedding Shape: torch.Size([128000, 1, 3072])
Layer 1 Statistics:
  Mean: -0.2013
  Std Dev: 11.7734
  Min: -329.5000
  Max: 359.7500
  Shape: torch.Size([128000, 3072])



# **2. BERT Embedding**

In [51]:
# # -*- coding: utf-8 -*-
# import os
# import torch
# from transformers import BertTokenizer, BertModel
# from tqdm import trange

# def rep_extract(task, mode, device, sents, stack_labels, labels):
#     model_path = 'google-bert/bert-large-uncased'
#     tokenizer = BertTokenizer.from_pretrained(model_path)
#     model = BertModel.from_pretrained(model_path).to(device)
#     model.eval()

#     max_len = 512
#     sents_reps = []
#     step = 512
#     for idx in trange(0, len(sents), step):
#         idx_end = idx + step
#         if idx_end > len(sents):
#             idx_end = len(sents)

#         # sents와 stack_labels 결합
#         sents_batch = sents[idx: idx_end]
#         stack_labels_batch = stack_labels[idx: idx_end]
#         combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

#         # 토큰화 및 인코딩
#         sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
#         sents_batch_encoding = sents_batch_encoding.to(device)

#         with torch.no_grad():
#             batch_outputs = model(**sents_batch_encoding)
#             reps_batch = batch_outputs.pooler_output
#         sents_reps.append(reps_batch.cpu())
#     sents_reps = torch.cat(sents_reps)

#     labels = torch.tensor(labels)

#     print(sents_reps.shape)
#     print(labels.shape)
#     path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
#     if not os.path.exists(path):
#         os.makedirs(path)
#     torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
#     torch.save(labels, path + f'{mode}_labels.pt')

# if __name__ == '__main__':
#     cuda_no = 0  # GPU 번호
#     task = 'bert'  # Task 이름
#     device = f'cuda:{cuda_no}'

#     # 'train' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['train']['text']
#     stack_labels = combined_dataset['train']['stack_labels']
#     labels = combined_dataset['train']['label']
#     rep_extract(task, 'train', device, sents, stack_labels, labels)

#     # 'validation' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['validation']['text']
#     stack_labels = combined_dataset['validation']['stack_labels']
#     labels = combined_dataset['validation']['label']
#     rep_extract(task, 'validation', device, sents, stack_labels, labels)

#     # 'test' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['test']['text']
#     stack_labels = combined_dataset['test']['stack_labels']
#     labels = combined_dataset['test']['label']
#     rep_extract(task, 'test', device, sents, stack_labels, labels)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

torch.Size([128000, 1024])
torch.Size([128000])


100%|██████████| 37/37 [06:24<00:00, 10.40s/it]


torch.Size([18500, 1024])
torch.Size([18500])


100%|██████████| 21/21 [03:38<00:00, 10.41s/it]

torch.Size([10600, 1024])
torch.Size([10600])


In [12]:
# bert의 layer추가
# -*- coding: utf-8 -*-
import os
import torch
from transformers import BertTokenizer, BertModel
from tqdm import trange

def rep_extract(task, mode, device, sents, stack_labels, labels, max_len=512, step=32):
    model_id = 'google-bert/bert-large-uncased'

    tokenizer = BertTokenizer.from_pretrained(model_id)
    model = BertModel.from_pretrained(model_id, output_hidden_states=True).to(device)
    model.eval()

    # 최적 가중치 설정 (Trial 0에서 찾은 가중치 고정)
    layer_weights = torch.tensor([0.666, 0.721, 0.280], device=device, dtype=torch.float16)
    layer_weights = layer_weights / layer_weights.sum()  # 가중치 정규화

    sents_reps = []
    for idx in trange(0, len(sents), step):
        idx_end = min(idx + step, len(sents))

        # sents와 stack_labels 결합
        sents_batch = sents[idx: idx_end]
        stack_labels_batch = stack_labels[idx: idx_end] if stack_labels else [""] * (idx_end - idx)
        combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

        # 토큰화 및 인코딩
        sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True).to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            selected_layers = [-1, -2, -3]
            layer_embeddings = torch.stack([batch_outputs.hidden_states[layer][:, 0, :] for layer in selected_layers], dim=0)
            weighted_sum = (layer_embeddings * layer_weights.view(-1, 1, 1)).sum(dim=0)

            sents_reps.append(weighted_sum.unsqueeze(1))

        torch.cuda.empty_cache()

    sents_reps = torch.cat(sents_reps)
    labels_tensor = torch.tensor(labels)

    # 저장 경로 및 파일 저장
    path = f'/content/drive/MyDrive/ML/AI-algorithm/project2/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps, path + f'{mode}_sents.pt')
    torch.save(labels_tensor, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # CUDA 오류 디버깅 활성화
    cuda_no = 0  # GPU 번호
    task = 'bert'  # Task 이름
    device = f'cuda:{cuda_no}'
    max_len = 512
    step = 32

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['train']['text']
    stack_labels = combined_dataset['train']['stack_labels']
    labels = combined_dataset['train']['label']
    rep_extract(task, 'train', device, sents, stack_labels, labels)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['validation']['text']
    stack_labels = combined_dataset['validation']['stack_labels']
    labels = combined_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, stack_labels, labels)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['test']['text']
    stack_labels = combined_dataset['test']['stack_labels']
    labels = combined_dataset['test']['label']
    rep_extract(task, 'test', device, sents, stack_labels, labels)


[I 2024-10-28 11:19:02,168] A new study created in memory with name: no-name-ee1eb7f8-5747-44fb-9d43-aaaeb63b2eb1
100%|██████████| 32/32 [00:20<00:00,  1.54it/s]
[I 2024-10-28 11:19:22,986] Trial 0 finished with value: 0.0007190262549556792 and parameters: {'weight_0': 0.6664838670293016, 'weight_1': 0.7211738381200303, 'weight_2': 0.280043759476582}. Best is trial 0 with value: 0.0007190262549556792.
100%|██████████| 32/32 [00:20<00:00,  1.54it/s]
[I 2024-10-28 11:19:43,775] Trial 1 finished with value: 0.0006815070519223809 and parameters: {'weight_0': 0.6524975171658911, 'weight_1': 0.9047851561494604, 'weight_2': 0.3520363395295484}. Best is trial 0 with value: 0.0007190262549556792.
100%|██████████| 32/32 [00:20<00:00,  1.54it/s]
[I 2024-10-28 11:20:04,561] Trial 2 finished with value: -0.0007943626260384917 and parameters: {'weight_0': 0.20380954758604503, 'weight_1': 0.2266263909806523, 'weight_2': 0.48640520108596785}. Best is trial 0 with value: 0.0007190262549556792.
100%|███

# **3. Roberta Embedding**

In [13]:
# # -*- coding: utf-8 -*-
# import os
# import torch
# from transformers import RobertaTokenizer, RobertaModel
# from tqdm import trange

# def rep_extract(task, mode, device, sents, stack_labels, labels):
#     model_path = 'FacebookAI/roberta-large'
#     tokenizer = RobertaTokenizer.from_pretrained(model_path)
#     model = RobertaModel.from_pretrained(model_path).to(device)
#     model.eval()

#     max_len = 512
#     sents_reps = []
#     step = 512
#     for idx in trange(0, len(sents), step):
#         idx_end = idx + step
#         if idx_end > len(sents):
#             idx_end = len(sents)

#         # sents와 stack_labels 결합
#         sents_batch = sents[idx: idx_end]
#         stack_labels_batch = stack_labels[idx: idx_end]
#         combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

#         # 토큰화 및 인코딩
#         sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True)
#         sents_batch_encoding = sents_batch_encoding.to(device)

#         with torch.no_grad():
#             batch_outputs = model(**sents_batch_encoding)
#             reps_batch = batch_outputs.last_hidden_state[:, 0, :]  # CLS 토큰의 벡터를 사용
#         sents_reps.append(reps_batch.cpu())
#     sents_reps = torch.cat(sents_reps)

#     labels = torch.tensor(labels)

#     print(sents_reps.shape)
#     print(labels.shape)
#     path = f'/content/drive/MyDrive/ML/AI-algorithm/project/{task}/dataset_tensor/'
#     if not os.path.exists(path):
#         os.makedirs(path)
#     torch.save(sents_reps.to('cpu'), path + f'{mode}_sents.pt')
#     torch.save(labels, path + f'{mode}_labels.pt')

# if __name__ == '__main__':
#     cuda_no = 0  # GPU 번호
#     task = 'roberta'  # Task 이름
#     device = f'cuda:{cuda_no}'

#     # 'train' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['train']['text']
#     stack_labels = combined_dataset['train']['stack_labels']
#     labels = combined_dataset['train']['label']
#     rep_extract(task, 'train', device, sents, stack_labels, labels)

#     # 'validation' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['validation']['text']
#     stack_labels = combined_dataset['validation']['stack_labels']
#     labels = combined_dataset['validation']['label']
#     rep_extract(task, 'validation', device, sents, stack_labels, labels)

#     # 'test' 데이터셋에 대한 임베딩 추출
#     sents = combined_dataset['test']['text']
#     stack_labels = combined_dataset['test']['stack_labels']
#     labels = combined_dataset['test']['label']
#     rep_extract(task, 'test', device, sents, stack_labels, labels)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 250/250 [46:50<00:00, 11.24s/it]


torch.Size([128000, 1024])
torch.Size([128000])


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 37/37 [06:52<00:00, 11.15s/it]


torch.Size([18500, 1024])
torch.Size([18500])


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 21/21 [03:56<00:00, 11.24s/it]

torch.Size([10600, 1024])
torch.Size([10600])


In [16]:
# -*- coding: utf-8 -*-
import os
import torch
import optuna
from transformers import RobertaTokenizer, RobertaModel
from tqdm import trange

def rep_extract(task, mode, device, sents, stack_labels, labels, max_len=512, step=32):
    model_id = 'FacebookAI/roberta-large'

    tokenizer = RobertaTokenizer.from_pretrained(model_id)
    model = RobertaModel.from_pretrained(model_id, output_hidden_states=True).to(device)
    model.eval()

    # 최적의 가중치를 찾기 위한 샘플링 데이터셋
    sample_size = 1000  # 샘플링 크기 설정
    sample_sents = sents[:sample_size]
    sample_stack_labels = stack_labels[:sample_size] if stack_labels else [""] * sample_size

    # 최적의 가중치 찾기 함수 정의
    def objective(trial):
        layer_weights = torch.tensor(
            [trial.suggest_float(f"weight_{i}", 0.1, 1.0) for i in range(3)],
            device=device,
            dtype=torch.float16
        )
        layer_weights = layer_weights / layer_weights.sum()

        sents_reps = []
        selected_layers = [-1, -2, -3]
        for idx in trange(0, len(sample_sents), step):
            idx_end = min(idx + step, len(sample_sents))

            sents_batch = sample_sents[idx: idx_end]
            stack_labels_batch = sample_stack_labels[idx: idx_end]
            combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

            sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True).to(device)

            with torch.no_grad():
                batch_outputs = model(**sents_batch_encoding)
                layer_embeddings = torch.stack([batch_outputs.hidden_states[layer][:, 0, :] for layer in selected_layers], dim=0)
                weighted_sum = (layer_embeddings * layer_weights.view(-1, 1, 1)).sum(dim=0)

                sents_reps.append(weighted_sum.unsqueeze(1))

            torch.cuda.empty_cache()

        sents_reps = torch.cat(sents_reps)
        return sents_reps.mean().item()

    # Optuna를 통한 최적 가중치 찾기
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

    # 최적화된 가중치로 임베딩 생성
    best_layer_weights = study.best_params
    layer_weights = torch.tensor(
        [best_layer_weights[f"weight_{i}"] for i in range(3)],
        device=device,
        dtype=torch.float16
    )
    layer_weights = layer_weights / layer_weights.sum()

    # 최적 가중치로 전체 데이터셋에 대해 임베딩 생성
    sents_reps = []
    selected_layers = [-1, -2, -3]  # 여기에 selected_layers를 추가하여 정의
    for idx in trange(0, len(sents), step):
        idx_end = min(idx + step, len(sents))

        sents_batch = sents[idx: idx_end]
        stack_labels_batch = stack_labels[idx: idx_end] if stack_labels else [""] * (idx_end - idx)
        combined_inputs = [f"{text} | Sub-label: {stack_label}" if stack_label else text for text, stack_label in zip(sents_batch, stack_labels_batch)]

        sents_batch_encoding = tokenizer(combined_inputs, return_tensors='pt', max_length=max_len, padding="max_length", truncation=True).to(device)

        with torch.no_grad():
            batch_outputs = model(**sents_batch_encoding)
            layer_embeddings = torch.stack([batch_outputs.hidden_states[layer][:, 0, :] for layer in selected_layers], dim=0)
            weighted_sum = (layer_embeddings * layer_weights.view(-1, 1, 1)).sum(dim=0)

            sents_reps.append(weighted_sum.unsqueeze(1))

        torch.cuda.empty_cache()

    sents_reps = torch.cat(sents_reps)
    labels_tensor = torch.tensor(labels)

    # 저장 경로 및 파일 저장
    path = f'/content/drive/MyDrive/ML/AI-algorithm/project2/{task}/dataset_tensor/'
    if not os.path.exists(path):
        os.makedirs(path)
    torch.save(sents_reps, path + f'{mode}_sents.pt')
    torch.save(labels_tensor, path + f'{mode}_labels.pt')

if __name__ == '__main__':
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # CUDA 오류 디버깅 활성화
    cuda_no = 0  # GPU 번호
    task = 'roberta'  # Task 이름
    device = f'cuda:{cuda_no}'
    max_len = 512
    step = 32

    # 'train' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['train']['text']
    stack_labels = combined_dataset['train']['stack_labels']
    labels = combined_dataset['train']['label']
    rep_extract(task, 'train', device, sents, stack_labels, labels)

    # 'validation' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['validation']['text']
    stack_labels = combined_dataset['validation']['stack_labels']
    labels = combined_dataset['validation']['label']
    rep_extract(task, 'validation', device, sents, stack_labels, labels)

    # 'test' 데이터셋에 대한 임베딩 추출
    sents = combined_dataset['test']['text']
    stack_labels = combined_dataset['test']['stack_labels']
    labels = combined_dataset['test']['label']
    rep_extract(task, 'test', device, sents, stack_labels, labels)


Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-10-28 12:39:40,937] A new study created in memory with name: no-name-604e385f-e48d-4857-b6c5-d44139aa9fd2
100%|██████████| 32/32 [00:23<00:00,  1.34it/s]
[I 2024-10-28 12:40:04,764] Trial 0 finished with value: -0.035632673650979996 and parameters: {'weight_0': 0.6021129388632425, 'weight_1': 0.23518820481396804, 'weight_2': 0.6146808849374658}. Best is trial 0 with value: -0.035632673650979996.
100%|██████████| 32/32 [00:23<00:00,  1.39it/s]
[I 2024-10-28 12:40:27,875] Trial 1 finished with value: -0.033025674521923065 and parameters: {'weight_0': 0.3965340461363184, 'weight_1': 0.5201506085073441, 'weight_2': 0.19474754318314946}. Best is trial 1 with value: -0.03302567452192306

In [11]:
import torch

# 파일 경로 설정
l_path = '/content/drive/MyDrive/ML/AI-algorithm/project2/llama3-2/dataset_tensor/'
b_path = '/content/drive/MyDrive/ML/AI-algorithm/project2/bert/dataset_tensor/'
r_path = '/content/drive/MyDrive/ML/AI-algorithm/project2/roberta/dataset_tensor/'
mode = 'train'  # train, test, validation 등의 모드에 따라 변경 가능

# 각 pt 파일의 임베딩 로드 및 크기 확인
l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
b_sents_reps = torch.load(b_path + f'{mode}_sents.pt')
r_sents_reps = torch.load(r_path + f'{mode}_sents.pt')

print(f"L Embedding Shape: {l_sents_reps.shape}")
print(f"B Embedding Shape: {b_sents_reps.shape}")
print(f"R Embedding Shape: {r_sents_reps.shape}")


<ipython-input-11-6fbeea2f5961>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  l_sents_reps = torch.load(l_path + f'{mode}_sents.pt')
<ipython-input-11-6fbeea2f5961>:11: 

L Embedding Shape: torch.Size([128000, 1, 3072])
B Embedding Shape: torch.Size([128000, 1, 1024])
R Embedding Shape: torch.Size([128000, 1, 1024])


In [8]:
import torch
import torch.nn.functional as F
import os

def normalize_and_save_embeddings(input_path, output_path, modes=['train', 'validation', 'test']):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    for mode in modes:
        # 임베딩 및 레이블 파일 로드
        embeddings_l = torch.load(f'{input_path}/llama3-2/dataset_tensor/{mode}_sents.pt')
        embeddings_b = torch.load(f'{input_path}/bert/dataset_tensor/{mode}_sents.pt')
        embeddings_r = torch.load(f'{input_path}/roberta/dataset_tensor/{mode}_sents.pt')
        labels = torch.load(f'{input_path}/llama3-2/dataset_tensor/{mode}_labels.pt')

        # 임베딩 정규화
        embeddings_l = F.normalize(embeddings_l, p=2, dim=-1)
        embeddings_b = F.normalize(embeddings_b, p=2, dim=-1)
        embeddings_r = F.normalize(embeddings_r, p=2, dim=-1)

        # 정규화된 임베딩 저장
        torch.save(embeddings_l, f'{output_path}/llama_{mode}_sents_normalized.pt')
        torch.save(embeddings_b, f'{output_path}/bert_{mode}_sents_normalized.pt')
        torch.save(embeddings_r, f'{output_path}/roberta_{mode}_sents_normalized.pt')
        torch.save(labels, f'{output_path}/{mode}_labels.pt')


normalize_and_save_embeddings(
    input_path='/content/drive/MyDrive/ML/AI-algorithm/project2',
    output_path='/content/drive/MyDrive/ML/AI-algorithm/project2/normalized'
)


<ipython-input-8-40973f8fb0bb>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings_l = torch.load(f'{input_path}/llama3-2/dataset_tensor/{mode}_sents.pt')
<ipython-

In [16]:
from torch.utils.data import Dataset, DataLoader

class UnifiedDataset(Dataset):
    def __init__(self, l_path, b_path, r_path, mode, max_length=512):
        self.l_sents_reps = torch.load(l_path + f'/llama_{mode}_sents_normalized.pt')  # [128000, 3072]
        self.b_sents_reps = torch.load(b_path + f'/bert_{mode}_sents_normalized.pt')   # [128000, 1024]
        self.r_sents_reps = torch.load(r_path + f'/roberta_{mode}_sents_normalized.pt') # [128000, 1024]
        self.labels = torch.load(l_path + f'/{mode}_labels.pt')
        self.sample_num = self.labels.shape[0]
        self.max_length = max_length

        # 임베딩 차원 출력
        print("Llama Embedding Shape:", self.l_sents_reps.shape)
        print("BERT Embedding Shape:", self.b_sents_reps.shape)
        print("RoBERTa Embedding Shape:", self.r_sents_reps.shape)
        print("Labels Shape:", self.labels.shape)

    def __getitem__(self, index):
        l_embedding = self.l_sents_reps[index].mean(dim=0)  # Mean pooling 적용
        b_embedding = self.b_sents_reps[index]
        r_embedding = self.r_sents_reps[index]
        label = self.labels[index]
        return l_embedding, b_embedding, r_embedding, label

    def __len__(self):
        return self.sample_num


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel
from torchviz import make_dot
from torchsummary import summary
import hiddenlayer as hl
import wandb

class UnifiedModel(nn.Module):
    def __init__(self, class_num, model_checkpoint, embedding_dim=3072, hidden_size=1024, output_size=256, dropout_rate=0.5, sigma=0.1):
        super(UnifiedModel, self).__init__()
        self.sigma = sigma
        # compress_layers의 입력 크기를 embedding_dim(3072)로 설정하고, 출력 크기를 hidden_size(1024)로 설정
        self.compress_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(embedding_dim, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ) for _ in range(3)
        ])

        # B와 R 임베딩을 3072 차원으로 맞추는 Linear 레이어 추가
        self.b_linear = nn.Linear(1024, 3072)
        self.r_linear = nn.Linear(1024, 3072)

        # EFT 기반 feature extractor
        self.language_model = AutoModel.from_pretrained(model_checkpoint)
        self.classifier_head = ExtremeFineTuningClassificationHead(
            input_size=hidden_size,
            hidden_size=output_size,
            output_size=class_num,
            activation_function=nn.PReLU(init=0.5),
            dropout=nn.Dropout(dropout_rate)
        )

        # fc1의 입력 크기를 hidden_size * 3으로 설정하여 compress_layers 출력과 맞춥니다.
        self.fc1 = nn.Linear(hidden_size * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Linear(output_size, class_num)
        #self.softmax = nn.Softmax(dim=1)

        # I-ELM 모드 플래그 설정
        self.elm_mode = False

    def forward(self, input_l, input_b, input_r):
        #batch_size = input_l.shape[0]

        # 입력 텐서를 float32로 변환
        input_l, input_b, input_r = input_l.float(), input_b.float(), input_r.float()

        # B와 R 임베딩을 3072 차원으로 변환
        input_b = self.b_linear(input_b)
        input_r = self.r_linear(input_r)

        # LLMEmbed 다중 임베딩 처리
        #embedded_inputs = [layer(embed) for layer, embed in zip(self.compress_layers, [input_l, input_b, input_r])]
            # LLMEmbed 다중 임베딩 처리
        embedded_inputs = []
        for layer, embed in zip(self.compress_layers, [input_l, input_b, input_r]):
            embed_processed = layer(embed)  # (batch_size, hidden_size)
            if embed_processed.dim() == 2:  # 2차원인 경우 배치 차원 유지
                embed_processed = embed_processed.unsqueeze(1)
            embedded_inputs.append(embed_processed)
        embedded_inputs = torch.cat(embedded_inputs, dim=-1)

        # 통합 입력으로 결합
        x = self.fc1(embedded_inputs)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        #return self.softmax(x)
        return x

    def set_elm_mode(self):
        """I-ELM 모드를 활성화합니다."""
        self.elm_mode = True

    def unset_elm_mode(self):
        """I-ELM 모드를 비활성화합니다."""
        self.elm_mode = False

    def step_helper(self, input_l, input_b, input_r, labels):
        """I-ELM용 입력 데이터 전처리"""
        combined_features = self.forward(input_l, input_b, input_r)
        labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=self.fc3.out_features).float()
        return combined_features, labels_one_hot
    #@torch.jit.script_method
    # def step_helper(input_l, input_b, input_r, labels, model):
    #     """I-ELM용 JIT 컴파일된 입력 데이터 전처리"""
    #     combined_features = model(input_l, input_b, input_r)
    #     labels_one_hot = F.one_hot(labels, num_classes=model.fc3.out_features).float()
    #     return combined_features, labels_one_hot

    # def _iterative_elm(self, input_features, label_one_hot):
    #     """I-ELM을 사용하여 가중치 업데이트 수행"""
    #     if not hasattr(self, 'H'):
    #         self.H = input_features.T @ input_features
    #         self.T = input_features.T @ label_one_hot
    #         self.output_weights = torch.linalg.pinv(self.H) @ self.T
    #     else:
    #         self.H += input_features.T @ input_features
    #         self.T += input_features.T @ label_one_hot
    #         self.output_weights = torch.linalg.pinv(self.H) @ self.T
    def _iterative_elm(self, input_features, label_one_hot):
        """I-ELM을 사용하여 가중치 업데이트 수행"""

        # input_features와 label_one_hot의 크기 출력
        #print(f"input_features shape: {input_features.shape}")
        #print(f"label_one_hot shape: {label_one_hot.shape}")
        input_features = input_features.squeeze(1)  # (32, 5)로 변환
        # 첫 번째 가중치 업데이트
        if not hasattr(self, 'H'):
            #print("Initializing H and T for the first time")  # 초기화 확인
            self.H = input_features.T @ input_features  # (feature_dim, feature_dim)
            self.T = input_features.T @ label_one_hot  # (feature_dim, num_classes)
            self.output_weights = torch.linalg.pinv(self.H) @ self.T
        else:
            # 기존의 H와 T에 새로운 배치 데이터 업데이트
            #print("Updating H and T with new batch data")
            # 기존의 H와 T에 새로운 배치 데이터 업데이트
            self.H += input_features.T @ input_features  # (feature_dim, feature_dim)
            self.T += input_features.T @ label_one_hot  # (feature_dim, num_classes)
            self.output_weights = torch.linalg.pinv(self.H) @ self.T
        # output_weights의 크기와 평균값을 출력하여 업데이트 확인
        #print(f"output_weights shape: {self.output_weights.shape}")
        #print(f"output_weights mean: {self.output_weights.mean().item():.4f}")


In [18]:
class ExtremeFineTuningClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_function=None, dropout=None):
        super(ExtremeFineTuningClassificationHead, self).__init__()
        self.input_weights = nn.Linear(input_size, hidden_size)
        self.output_weights = nn.Linear(hidden_size, output_size)
        self.activation_function = activation_function if activation_function else nn.Tanh()
        self.dropout = dropout

    def forward(self, features):
        x = features
        if self.dropout:
            x = self.dropout(x)
        x = self.input_weights(x)
        x = self.activation_function(x)
        if self.dropout:
            x = self.dropout(x)
        x = self.output_weights(x)
        return x

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

# 하이퍼파라미터를 포함한 설정 딕셔너리
config = {
    'eft': {  # Extreme Fine-Tuning 관련 설정
        'hidden_size': 1024,
        'output_size': 256,
        'dropout_rate': 0.1,
        'learning_rate': 1e-4  # EFT 학습률
    },
    'llmembed': {  # LLM 임베딩 관련 설정
        'sigma': 0.1,
        'dropout_rate': 0.5,
        'compress_layers': 5,
        'learning_rate': 5e-5  # LLM 임베딩 학습률
    },
    'training': {  # 학습 관련 설정
        'total_epochs': 20,
        'elm_epochs': 5,  # I-ELM 학습 수행 에폭 수
        'batch_size': 32,
        'num_classes': 5
    },
    'model': {  # 모델 체크포인트 및 경로 설정
        #'checkpoint': 'unsloth/Llama-3.2-3B-Instruct',
        'checkpoint' : 'unsloth/Llama-3.2-1B-Instruct',
        'embedding_paths': {
            'llama': '/content/drive/MyDrive/ML/AI-algorithm/project2/normalized',
            'bert': '/content/drive/MyDrive/ML/AI-algorithm/project2/normalized',
            'roberta': '/content/drive/MyDrive/ML/AI-algorithm/project2/normalized'
        }
    }
}


In [20]:
import torch
import torch.nn.functional as F

def train_with_ielm(model, dataloader, device, epoch):
    """I-ELM 모드 학습"""
    model.set_elm_mode()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_l, input_b, input_r, labels = batch
            input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

            # I-ELM 가중치 업데이트
            features, labels_one_hot = model.step_helper(input_l, input_b, input_r, labels)
            model._iterative_elm(features, labels_one_hot)
            # 손실 계산
            loss = F.mse_loss(features.squeeze(1), labels_one_hot.float())
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)

    # wandb에 I-ELM 손실 기록
    wandb.log({"ielm_loss": avg_loss, "epoch": epoch})
    print(f"I-ELM - Loss: {avg_loss:.4f}")
    model.unset_elm_mode()

def train_with_backprop(model, dataloader, device, loss_fn, optimizer, epoch):
    model.train()
    total_loss, total_acc = 0, 0
    for batch in dataloader:
        input_l, input_b, input_r, labels = batch
        #labels = labels.view(-1)  # (batch_size,)로 변환
        # labels가 2D 텐서인지 확인하고, 1D 정수 인덱스로 변환
        if labels.dim() == 2:
            labels = labels.argmax(dim=1)  # 원핫 인코딩을 정수 인덱스로 변환
        # labels의 shape 출력
        #print(f"Labels shape after transformation: {labels.shape}")  # Debug print

        input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_l, input_b, input_r).squeeze(1)  # (batch_size, 1, num_classes) -> (batch_size, num_classes)로 변환
        #print(f"Outputs shape: {outputs.shape}")  # Debug print
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total_acc += (preds == labels).sum().item() / len(labels)

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    # wandb에 손실과 정확도 기록
    wandb.log({"train_loss": avg_loss, "train_accuracy": avg_acc, "epoch": epoch})
    print(f"Backpropagation - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}")

    # 에폭 끝에서 캐시 비우기
    torch.cuda.empty_cache()



In [6]:
%pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
#unsloth 적용을 위한 코드
import torch
import torch.nn.functional as F
import unsloth

def train_with_ielm(model, dataloader, device, epoch):
    """I-ELM 모드 학습"""
    model.set_elm_mode()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_l, input_b, input_r, labels = batch
            input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

            # I-ELM 가중치 업데이트
            features, labels_one_hot = model.step_helper(input_l, input_b, input_r, labels)
            model._iterative_elm(features, labels_one_hot)
            # 손실 계산
            loss = F.mse_loss(features.squeeze(1), labels_one_hot.float())
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)

    # wandb에 I-ELM 손실 기록
    wandb.log({"ielm_loss": avg_loss, "epoch": epoch})
    print(f"I-ELM - Loss: {avg_loss:.4f}")
    model.unset_elm_mode()

def train_with_backprop_unsloth(model, dataloader, device, loss_fn, optimizer, epoch):
    model.train()
    total_loss, total_acc = 0, 0
    for batch in dataloader:
        input_l, input_b, input_r, labels = batch
        #labels = labels.view(-1)  # (batch_size,)로 변환
        # labels가 2D 텐서인지 확인하고, 1D 정수 인덱스로 변환
        if labels.dim() == 2:
            labels = labels.argmax(dim=1)  # 원핫 인코딩을 정수 인덱스로 변환
        # labels의 shape 출력
        #print(f"Labels shape after transformation: {labels.shape}")  # Debug print

        input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_l, input_b, input_r).squeeze(1)  # (batch_size, 1, num_classes) -> (batch_size, num_classes)로 변환
        #print(f"Outputs shape: {outputs.shape}")  # Debug print
        loss = loss_fn(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        total_acc += (preds == labels).sum().item() / len(labels)

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader)
    # wandb에 손실과 정확도 기록
    wandb.log({"train_loss": avg_loss, "train_accuracy": avg_acc, "epoch": epoch})
    print(f"Backpropagation with Unsloth - Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}")

    # 에폭 끝에서 캐시 비우기
    torch.cuda.empty_cache()



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [22]:
def train_model(model, dataloader, device, loss_fn, optimizer, config):
    epochs = config['training']['total_epochs']
    elm_epochs = config['training']['elm_epochs']

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        if epoch < elm_epochs:
            print("Using I-ELM training")
            train_with_ielm(model, dataloader, device, epoch)
        else:
            print("Using Backpropagation training")
            #train_with_backprop(model, dataloader, device, loss_fn, optimizer, epoch)
            train_with_backprop_unsloth(model, dataloader, device, loss_fn, optimizer, epoch)


In [23]:
def evaluate_model(model, dataloader, device):
    model.eval()
    total_acc, total_loss = 0, 0
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for input_l, input_b, input_r, labels in dataloader:
            input_l, input_b, input_r, labels = input_l.to(device), input_b.to(device), input_r.to(device), labels.to(device)

            # labels가 2D 텐서인 경우 1D로 변환
            if labels.dim() == 2:
                labels = labels.argmax(dim=1)

            outputs = model(input_l, input_b, input_r)
            outputs = outputs.squeeze(1)  # (batch_size, 1, num_classes) -> (batch_size, num_classes)로 변환

            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            total_acc += (preds == labels).sum().item()

    avg_loss = total_loss / len(dataloader)
    avg_acc = total_acc / len(dataloader.dataset)

    wandb.log({"test_loss": avg_loss, "test_accuracy": avg_acc})
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {avg_acc:.4f}")


In [24]:
# unsloth test 중
import unsloth

def create_unsloth_optimizer(model, lr=5e-5):
    """Unsloth 최적화 설정"""
    optimizer = unsloth.Sloth(
        model.parameters(),
        lr=lr,
        rho=0.05,
        weight_decay=0.01
    )
    return optimizer


In [25]:

if __name__ == '__main__':

    # wandb 초기화
    wandb.init(project="classification_finetuning", name="unified_model_run", config=config)
    wandb.config.update(config)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    train_dataset = UnifiedDataset(
        l_path=config['model']['embedding_paths']['llama'],
        b_path=config['model']['embedding_paths']['bert'],
        r_path=config['model']['embedding_paths']['roberta'],
        mode='train'
    )
    validation_dataset = UnifiedDataset(
        l_path=config['model']['embedding_paths']['llama'],
        b_path=config['model']['embedding_paths']['bert'],
        r_path=config['model']['embedding_paths']['roberta'],
        mode='validation'
    )
    test_dataset = UnifiedDataset(
        l_path=config['model']['embedding_paths']['llama'],
        b_path=config['model']['embedding_paths']['bert'],
        r_path=config['model']['embedding_paths']['roberta'],
        mode='test'
    )

    train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=config['training']['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config['training']['batch_size'], shuffle=False)

    model = UnifiedModel(
        class_num=config['training']['num_classes'],
        model_checkpoint=config['model']['checkpoint'],
        hidden_size=config['eft']['hidden_size'],
        output_size=config['eft']['output_size'],
        dropout_rate=config['eft']['dropout_rate'],
        sigma=config['llmembed']['sigma']
    ).to(device)

    #wandb.watch(model, log="all",log_freq=10)
    loss_fn = nn.CrossEntropyLoss()
    #optimizer = optim.AdamW(model.parameters(), lr=config['llmembed']['learning_rate'])
    #eft_optimizer = optim.AdamW(model.language_model.parameters(), lr=config['eft']['learning_rate'])

    # 언어 모델 파라미터와 나머지 파라미터를 분리하여 Optimizer 설정
    # optimizer = optim.AdamW([
    #     {'params': model.language_model.parameters(), 'lr': config['eft']['learning_rate']},
    #     {'params': [p for n, p in model.named_parameters() if not n.startswith('language_model')], 'lr': config['llmembed']['learning_rate']}
    # ])
    # unsloth 적용하기 위한 optimizer 초기화
    optimizer = create_unsloth_optimizer(model, lr=5e-5)



    def print_model_summary(model):
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Model Summary:\nTotal Parameters: {total_params}")
        for name, param in model.named_parameters():
            print(f"{name}: {param.size()}")

    # Training 시작 전에 모델 요약 정보 출력
    print_model_summary(model)

    # 학습 수행
    print("Training the model...")
    train_model(model, train_loader, device, loss_fn, optimizer, config)
    # 모델 평가
    print("Evaluating the model...")
    evaluate_model(model, test_loader, device)

    # wandb finish
    wandb.finish()

wandb: Currently logged in as: jeremy-su1 (ai_expert). Use `wandb login --relogin` to force relogin


<ipython-input-16-b4656475d875>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.l_sents_reps = torch.load(l_path + f'/llama_{mode}_sents_normalized.pt')  # [128000, 30

Llama Embedding Shape: torch.Size([128000, 1, 3072])
BERT Embedding Shape: torch.Size([128000, 1, 1024])
RoBERTa Embedding Shape: torch.Size([128000, 1, 1024])
Labels Shape: torch.Size([128000])
Llama Embedding Shape: torch.Size([18500, 1, 3072])
BERT Embedding Shape: torch.Size([18500, 1, 1024])
RoBERTa Embedding Shape: torch.Size([18500, 1, 1024])
Labels Shape: torch.Size([18500])
Llama Embedding Shape: torch.Size([10600, 1, 3072])
BERT Embedding Shape: torch.Size([10600, 1, 1024])
RoBERTa Embedding Shape: torch.Size([10600, 1, 1024])
Labels Shape: torch.Size([10600])


config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

AttributeError: module 'unsloth' has no attribute 'Sloth'

In [26]:
print(dir(unsloth))

['FastLanguageModel', 'FastLlamaModel', 'FastMistralModel', 'FastQwen2Model', 'PatchDPOTrainer', 'SUPPORTS_BFLOAT16', 'UnslothTrainer', 'UnslothTrainingArguments', 'Version', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'add_new_tokens', 'apply_chat_template', 'bnb', 'cdequantize_blockwise_fp32', 'chat_templates', 'check_tokenizer', 'cohere', 'create_huggingface_repo', 'dpo', 'fix_sentencepiece_gguf', 'fix_sentencepiece_tokenizer', 'gemma', 'gemma2', 'get_chat_template', 'importlib', 'inspect', 'is_bf16_supported', 'is_bfloat16_supported', 'kernels', 'keynames', 'libcuda_dirs', 'llama', 'load_correct_tokenizer', 'loader', 'major_torch', 'major_version', 'mapper', 'minor_torch', 'minor_version', 'mistral', 'models', 'np', 'old_is_bf16_supported', 'os', 'patch_saving_functions', 'print_quantization_methods', 'qwen2', 're', 'remove_special_tokens', 'save', 'save_to_gguf', 'standardize_sharegpt', 'subprocess', 'sys', 

# **5. Model operation (multi)**

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import numpy as np

# epoch - Train
def Train_multi(dataloader, device, model, loss_fn, optimizer):
    loss_list, acc_list, micro_f1_list, macro_f1_list = [], [], [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.train()
        pred = model(batch_l.float(), batch_b.float(), batch_r.float())
        loss = loss_fn(pred, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pred_y = torch.max(pred, 1).indices
        # print(pred_y)
        acc = accuracy_score(batch_y.cpu(), pred_y.cpu())
        micro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='micro')
        macro_f1 = f1_score(batch_y.cpu(), pred_y.cpu(), average='macro')
        loss = loss.cpu()

        loss_list.append(loss.item())
        acc_list.append(acc)
        micro_f1_list.append(micro_f1)
        macro_f1_list.append(macro_f1)
    print(f'loss: {np.mean(loss_list):.4f}')
    print(f'acc: {np.mean(acc_list):.4f}')
    print(f'micro_f1: {np.mean(micro_f1_list):.4f}')
    print(f'macro_f1: {np.mean(macro_f1_list):.4f}')

def Test_multi(dataloader, device, model, loss_fn):
    avg_loss = 0
    total_pred, total_y = [], []

    for batch_i, batch_loader in enumerate(tqdm(dataloader)):
        batch_l, batch_b, batch_r, batch_y = batch_loader
        batch_l, batch_b, batch_r, batch_y = batch_l.to(device), batch_b.to(device), batch_r.to(device), batch_y.to(device)

        model.eval()
        with torch.no_grad():
            pred = model(batch_l.float(), batch_b.float(), batch_r.float())
            loss = loss_fn(pred, batch_y)
            loss = loss.to('cpu')
            avg_loss += loss.item()

        pred_y = torch.max(pred, 1).indices
        total_pred.append(pred_y.cpu())
        total_y.append(batch_y.cpu())

    avg_loss = avg_loss / (batch_i+1)

    total_y = torch.cat(total_y)
    total_pred = torch.cat(total_pred)
    acc = accuracy_score(total_y, total_pred)
    micro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='micro')
    macro_f1 = f1_score(total_y.cpu(), total_pred.cpu(), average='macro')

    print(f'avg loss: {avg_loss:.4f}')
    print(f'acc: {acc:.4f}')
    print(f'micro_f1: {micro_f1:.4f}')
    print(f'macro_f1: {macro_f1:.4f}')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel
from transformers import AutoConfig

# Load the tokenizers and models for Llama2, BERT, and Roberta
llama2_tokenizer = AutoTokenizer.from_pretrained("daryl149/llama-2-7b-chat-hf", trust_remote_code=True)
llama2_tokenizer.pad_token = llama2_tokenizer.eos_token  # 패딩 토큰 설정
llama2_config = AutoConfig.from_pretrained("daryl149/llama-2-7b-chat-hf", output_hidden_states=True)
llama2_model = AutoModelForCausalLM.from_pretrained("daryl149/llama-2-7b-chat-hf", config=llama2_config)

bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
bert_model = BertModel.from_pretrained('google-bert/bert-large-uncased')

roberta_tokenizer = RobertaTokenizer.from_pretrained('FacebookAI/roberta-large')
roberta_model = RobertaModel.from_pretrained('FacebookAI/roberta-large')

# Make sure all models are in evaluation mode and moved to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llama2_model.eval().to(device)
bert_model.eval().to(device)
roberta_model.eval().to(device)

# Initialize the downstream model
class_num = 5  # For example, if you have 8 classes
SIGMA = 0.1  # SIGMA value for your downstream model
downstream_model = DownstreamModel(class_num, SIGMA).to(device)

model_load_path = "/content/drive/MyDrive/LLMEmbed/model_weights_agnews.pth"

# 가중치 로드
downstream_model.load_state_dict(torch.load(model_save_path, map_location=device))
downstream_model.eval()

def get_llama2_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Average the last 5 layers
        embedding = torch.stack([torch.mean(outputs.hidden_states[i], dim=1) for i in range(-1, -6, -1)], dim=1)
    return embedding

def get_bert_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use pooler_output for BERT embeddings
        embedding = outputs.pooler_output
    return embedding

def get_roberta_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        # Use the first token ([CLS] token) representation
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding

def infer(text, downstream_model, device):
    # 각 모델로부터 임베딩을 추출
    llama2_emb = get_llama2_embedding(text, llama2_tokenizer, llama2_model, device)
    bert_emb = get_bert_embedding(text, bert_tokenizer, bert_model, device)
    roberta_emb = get_roberta_embedding(text, roberta_tokenizer, roberta_model, device)

    # Forward pass through the downstream model
    with torch.no_grad():
        prediction = downstream_model(llama2_emb, bert_emb, roberta_emb)

    return prediction


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-dde813e42baf>:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com

# **9. 추론 테스트**

In [ ]:
label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech",
    4: "StackOverFlow"
}

# 주어진 텍스트를 토대로 예측을 5번 실행
texts = [
    "US trade deficit swells in June The US trade deficit has exploded 19 to a record \$55.8bn as oil costs drove imports higher, according to a latest figures.",
    "Indians Beat Twins 7-1, Nearing AL Lead (AP) AP - The Cleveland Indians pulled within one game of the AL Central lead, scoring four runs in the first inning and beating the Minnesota Twins 7-1 Saturday night behind home runs by Travis Hafner and Victor Martinez.",
    "Pilgrims Crowd Field for Mass With Pope LOURDES, France - A frail Pope John Paul II celebrated an open-air Mass on Sunday as several hundred thousand pilgrims, many in wheelchairs, crowded onto a field near a French shrine to the Virgin Mary that is associated with miraculous cures of the sick. The Mass was a highlight of the Pope's two-day visit to Lourdes, a town in the Pyrenees where Roman Catholic tradition says St...",
    "T. Rex Had Teen Growth Spurt, Scientists Say (Reuters) Reuters - Tyrannosaurus Rex grew incredibly fast\during a teenaged growth spurt that saw the dinosaur expand its\bulk by six times, but the fearsome beasts \"lived fast and died\young,\" researchers said on Wednesday.",
    "working of compareTo() method of Comparable interface            I have one Employee class and the requirement is to sort the objects using comparable interface. The output with this code is :The difference of this id and other id is..** 6  other id**1The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**6The difference of this id and other id is..** 3  other id**1The difference of this id and other id is..** 11  other id**3The difference of this id and other id is..** 11  other id**6",
]

# 각 텍스트에 대해 예측 실행 및 레이블 출력
for i, text in enumerate(texts):
    prediction = infer(text, downstream_model, device)
    predicted_label = torch.argmax(prediction, dim=1).item()
    predicted_score = torch.max(prediction).item()
    label_name = label_map[predicted_label]
    print(f"Text {i+1} Predicted label: {label_name}, Score: {predicted_score:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text 1 Predicted label: Business, Score: 1.0000
Text 2 Predicted label: Sports, Score: 0.9954
Text 3 Predicted label: World, Score: 1.0000
Text 4 Predicted label: Sci/Tech, Score: 1.0000
Text 5 Predicted label: StackOverFlow, Score: 1.0000
